# Install required libraries and imports.

In [1]:
!pip install -U transformers
!pip3 install torch --index-url https://download.pytorch.org/whl/cu118
!pip install -U datasets
from datasets import load_dataset, DatasetDict
import os

# Allocate maximum CUDA memory reserve in an attempt to prevent CUDA out of memory errors
# Reserve is simply the reserved memory, not the in-use memory.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"

Looking in indexes: https://download.pytorch.org/whl/cu118
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/ec/93/454ada0d1b289a0f4a86ac88dbdeab54921becabac45da3da787d136628f/datasets-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successful

Tell PyTorch to use GPU wherever possible

In [2]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Data processing

First, the dataset and checkpoint needs to be initialised.

### Dataset location
Dataset location depends on where notebook is running, for ease I set it up to just uncomment line below depending on location the notebook is running as I run the notebook in a lot of locations.

In [3]:
dataset_location = "/kaggle/input/new-datasets/Terms_dataset.jsonl" # Kaggle

#dataset_location = "Privacy_Policy_dataset.jsonl" # Local / Google Colab

### Initialise dataset
To initialise the dataset I use the "dataset" library from python.

I split the dataset into three sets:
- Training set - The data shown to the model during training
- Validation - The data shown to the model to calculate loss on backward pass
- Test - Reserved strictly for after the model is trained, used to evaluate the model on a completely unseen set

However, the "datasets" library doesn't offer the possibility to split into three sets so I use a workaround sourced from: [This stackoverflow post](https://stackoverflow.com/questions/76001128/splitting-dataset-into-train-test-and-validation-using-huggingface-datasets-fun)

It works by first splitting the data set into a train set (80%) and a validation set (20%).

It then splits this validation set into a train set and validation set of 50% each, resulting in two sets of 10% each.

A final dataset consisting of a train, test and validation set is then built using these split datasets

In [4]:
dataset = load_dataset("json", data_files=dataset_location,split='train')

dataset = dataset.shuffle(seed=2424)

test_valid_split_dataset = dataset.train_test_split(test_size=0.2, shuffle=False)

test_split = test_valid_split_dataset['test'].train_test_split(test_size=0.5, shuffle = False)

dataset = DatasetDict({
    'train': test_valid_split_dataset['train'],
    'test': test_split['test'],
    'valid': test_split['train']})

print(dataset)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['summary', 'document'],
        num_rows: 417
    })
    test: Dataset({
        features: ['summary', 'document'],
        num_rows: 53
    })
    valid: Dataset({
        features: ['summary', 'document'],
        num_rows: 52
    })
})


### Reviewing the dataset
Next I want to see the properties of the dataset, to understand what i'm working with.

For training a summarisation model knowing the length of the collected documents is crucial.

The largest summarisation base model is only capable of processing 16384 tokens - higher token limits is a limitation in NLP as a whole.

Roughly, [One token is equal to about four English characters](https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them)

This gives roughly 65,536 characters which the model will be able to parse and create a summary for, and anything which exceeds this number needs to be truncated down to the 16384 token limit.

This unfortunately means on some documents, some detail will be missing.

However, as visible below, the average document in the train set is just 29,962 characters, considerably under the maximum token limit, thus, for most items in the dataset this isn't a problem.

In [5]:
length_of_first_item = len(dataset['train'][0]['document'])
print(f'The length of the first document in the train dataset is {length_of_first_item} characters')

length_of_longest_document = len(max(dataset['train'], key=lambda x: len(x['document']))['document'])
print(f'Length of the longest document in the train dataset is {length_of_longest_document} characters')

length_of_shortest_document = len(min(dataset['train'], key=lambda x: len(x['document']))['document'])
print(f'Length of the shortest document in the train dataset is {length_of_shortest_document} characters')

total_char_count = sum(map(len, dataset['train']['document']))
avg_char_count = round(total_char_count / len(dataset['train']['document']))

print(f'The average character count in the dataset is: {avg_char_count}')

The length of the first document in the train dataset is 111553 characters
Length of the longest document in the train dataset is 247686 characters
Length of the shortest document in the train dataset is 1472 characters
The average character count in the dataset is: 29962


# Base model

I will be utilising transfer learning to train a model.

This takes the base of a model trained on some other task but in a similar domain (e.g. summarising books), removes the head of the model which is more specialised (e.g. contains information specific to books), while retaining useful information about the English language. The model is then trained on a new specific task, in my case, summarising terms and conditions or privacy policies, utilising its pre-existing knowledge of the English language.

This significantly reduces training time and resources required for training such that I can stay within the final year project deadlines.

The model I will be using as a base is the [led-large-book-summary](https://huggingface.co/pszemraj/led-large-book-summary). This model utilises the Longformer Encoder-Decoder (LED) model as it's base, and was trained further to summarise long-form text such as novels, plays and stories from the [BookSum dataset](https://arxiv.org/abs/2105.08209)

 Below I initialise the tokeniser for this model through the [Hugging Face](https://huggingface.co/models) library, which offer a variety of base models for transfer-learning. 
  - This base model is ~1.5 GB 


In [6]:
from transformers import AutoTokenizer

base_model_name = "pszemraj/led-large-book-summary"
tokeniser = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Transformer models *only* take numerical inputs, thus, a tokeniser is responsible for transforming the input text into its numerical representation.

It does this by splitting text into "tokens" which are small groups of characters.

Tokenisers also use special characters, often indicating the start and the end of sequences and words.

Below I display an example by tokenising a simple string using the tokeniser for the "led-large-book-summary" model

The `input_ids` represents the tokenised input, `attention_mask` is tells the model to ignore tokens if the equivalent index in the attention mask array is zero.

Note when converting the string back to English, we can see`<s>` being used to indicate the start of a sequence, and each word beginning with `Ġ`.
 - This changes depending on the base model, but the Hugging Face library picks out the right tokeniser for the base model.

Also, words can be split into two tokens if it is deemed useful, below `tokeniser` is split into two tokens `token` and `iser`, as the tokens `iser` and `token` could later be re-used with other words, saving tokens.

In [7]:
test_string = tokeniser("This is a test string to test out the tokeniser")
print(test_string)
tokeniser.convert_ids_to_tokens(test_string.input_ids)

{'input_ids': [0, 713, 16, 10, 1296, 6755, 7, 1296, 66, 5, 19233, 5999, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


['<s>',
 'This',
 'Ġis',
 'Ġa',
 'Ġtest',
 'Ġstring',
 'Ġto',
 'Ġtest',
 'Ġout',
 'Ġthe',
 'Ġtoken',
 'iser',
 '</s>']

Next, a function is needed which will tokenise the text.

Here, the maximum length can be defined as 16384 tokens and the tokeniser will be responsible for ensuring any text exceeding this is truncated.

In the dataset, the "document" column contains the input document (privacy policy or Terms of Service)

The "summary" column contains the ground truth summary for the matching document.
 - This doesn't need to be truncated, as they are all <500 tokens

Furthermore, this function assigns the tokenised ground truth summaries `input_id`'s to the "labels" property of the tokenised documents.
 - This is the format required for training a summarisaiton model in pytorch.

In [8]:
def tokenise_truncate_dataset(input):
    truncated_input = tokeniser(
        input["document"],
        max_length = 16384,
        truncation = True
    )
    labels = tokeniser( # don't truncate labels
        input["summary"],
        truncation = False,
    )
    truncated_input["labels"] = labels["input_ids"]
    return truncated_input
# By passing `batched = true` into the map function, more than one item is applied to the function at a time.
tokenised_dataset = dataset.map(tokenise_truncate_dataset, batched = True)

Map:   0%|          | 0/417 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

Map:   0%|          | 0/52 [00:00<?, ? examples/s]

The properties of the dataset can now be viewed, as expected, there is a train, test and validation dataset.

In [9]:
tokenised_dataset

DatasetDict({
    train: Dataset({
        features: ['summary', 'document', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 417
    })
    test: Dataset({
        features: ['summary', 'document', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 53
    })
    valid: Dataset({
        features: ['summary', 'document', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 52
    })
})

However, the "summary" and "document" dataset are no longer needed, as we have their tokenised equivalents - "input_ids" and "labels".

Thus, these can be removed.

Furthermore, the dataset needs to be set to return pytorch tensors, in order to be able to be trained in pytorch.

In [10]:
tokenised_dataset = tokenised_dataset.remove_columns(["summary","document"])
tokenised_dataset.set_format("torch")
tokenised_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 417
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 53
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 52
    })
})

Next, a data collator needs to be defined.

(Below Information sourced from: https://huggingface.co/docs/transformers/main_classes/data_collator)

This is responsible for constructing batches and applying pre-processing such as padding to ensure all inputs are of the same size.

The Hugging Face library provides a function for sourcing a data collator with padding.

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokeniser, model= base_model_name)


# Model

First, the base model needs to be defined.

Next, an optimiser needs to be defined.

The standard optimiser to use is `adamW`, but again, due to VRAM limitations, a less memory-intensive optimiser will be used called "adafactor"

In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM
from transformers.optimization import Adafactor

base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

Next, ROUGE needs to be defined for the evaluation loop, and a function needs to be defined which will compute the rouge scores.

The rouge score metric is imported from the `rouge_score` python library, which will calculate and return the following metrics
 
 ### ROUGE-1
 ROUGE-1 is the overlap of words between the produced summary and ground truth summary.
 $$\frac{overlapping \space words}{total \space words}$$
 
 ### ROUGE-2
 ROUGE-2 is the overlap of bi-grams (pairs of words)
 
 $$\frac{overlapping \space pairs}{total \space number \space of \space bi-grams}$$ 
 
 ### ROUGE-L
 Source partially from (https://en.wikipedia.org/wiki/Longest_common_subsequence)
 
 ROUGE-L is based on the idea of a *longest common subsequence* - the longest common sequence of words between two texts.
 
 A subsequence does not necessarily have to be *consecutive*, letters can be skipped to make a subsequence.
 
 Consider strings "ABCD" and "ACBAD" - the longest common subseqeunces sequences are ABD and ACD.
 
 $$\frac{Longest \space common \space subsequence}{total \space words}$$ 
 
 Such that for the above example, if we consider "ABCD" the produced text and "ACBAD" the ground truth, ROUGE-L is equal to $ \frac{3}{5} = 0.6$
 
 ### ROUGE-L-SUM
 sourced from https://dev.to/aws-builders/mastering-rouge-matrix-your-guide-to-large-language-model-evaluation-for-summarization-with-examples-jjg
 
This is similar to ROUGE-L, but instead compares this at a *sentence* level, calculating ROUGE-L for each sentence in the produced summary.

This is a better measurement of accuracy in my use-case, as the produced summaries are split into sentences, where each sentence is a "summary point".

In [13]:
!pip install evaluate
!pip install rouge_score

import evaluate
import numpy as np

rouge = evaluate.load("rouge")

def decode_and_find_rouge(result):
    predictions,labels = result # extract predictions and labels from the passed in result.

    # Decode by converting tokenised inputs back into English representations
    decoded_predictions = tokeniser.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokeniser.batch_decode(labels, skip_special_tokens = True)
    
    # Calculate rouge scores by passing in predictions and ground truths
    rouge_score = rouge.compute(predictions=decoded_predictions, references=decoded_labels)
    
    # Calculate the average generated length fo examples.
    num_of_predictions = [np.count_nonzero(pred != tokeniser.pad_token_id) for pred in predictions]
    rouge_score["gen_len"] = np.mean(num_of_predictions)

    # Round and return results.
    return {k: round(v, 4) for k, v in rouge_score.items()}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=21deca0b10972b60fc2adf7b824f03382397aa43031a9829dc1fa1247bb8acd2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


Next, the model training loop needs to be defined:

This is a very expensive model on memory and it is essential to avoid exceeding the GPU's VRAM, thus the hyper-parameters are configured as such:

1) The batch size will be set to just 1 to reduce the amount of data stored in memory.

2) 16-bit floating point precision will be used, rather than 32-bit.

3) The optimiser "adafactor" is used over the industry standard of "Adam". As a result, the model converges slower but also uses less memory.

4) "Gradient Checkpointing" is used, instructing the model to forget the majority of forward-pass activations and instead recompute them on demand during the backward pass, saving only the "most important" activations. Significantly slower training time, but also saves a lot of VRAM.

5) "Gradient Accumulation" is used simulate a "larger" effective batch size. Instead of updating the model's parameters after processing eachbatch, the gradients are accumulated over 2 batches before performing a single update.

In [14]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

models_arguments = Seq2SeqTrainingArguments(
    output_dir="new_terms_model_4_epoch",
    evaluation_strategy="epoch", # Run evaluation function on each epoch
    learning_rate=2e-5, # learning rate hyperparameter set to 0.00002
    per_device_train_batch_size= 1, # split into batches of 1 for training
    per_device_eval_batch_size=1, # split to batches of 1 for evaluation
    weight_decay=0.01, # Utilises L2 regularization in an attempt to prevent overfitting
    save_total_limit=3, # save 3 checkpoints only and delete older checkpoints (Kept using all RAM without)
    num_train_epochs=4, # train for 4 epochs
    predict_with_generate=True, # Generate summaries for each input ; essential for summarisation tasks
    fp16=True, # use 16 bit floating point - reduced memory usage
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adafactor"
)

# Collect the previously defined trainer parameters, such as the evaluation technique, tokeniser and datasets.
trainer = Seq2SeqTrainer(
    model=base_model,
    args=models_arguments,
    train_dataset=tokenised_dataset["train"],
    eval_dataset=tokenised_dataset["valid"],
    tokenizer=tokeniser,
    data_collator=data_collator,
    compute_metrics=decode_and_find_rouge,
)

import os

# Uncomment below line if using Kaggle.
os.environ["WANDB_DISABLED"] = "true"

trainer.train()
trainer.save_model("new_terms_model_4_epoch")

You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,0.596928,0.401200,0.249500,0.250800,0.250600,114.846200
2,0.741200,0.500441,0.475500,0.299200,0.262600,0.262900,158.250000
3,0.741200,0.487602,0.487400,0.320200,0.267800,0.268000,170.173100


# Evaluation

## Evaluating tokeniser

First, I want to evaluate the tokeniser to see if a lot of text is being truncated and lost during the tokenisation process

From the example below we can see some characters are lost, however, on review it appears that it is just removing excessive text that existed in the input text, a specific example is below:

`For the purposes of the , is the "data controller".`

became:

`For the purposes of the, is the "data controller".`

Note the absence of the extra space before the comma which existed in the un-tokenised data.

also note the sentence appears to be missing a word - this is a side effect of webscraping, which missed a very minor amount of text while collecting the dataset, such as perhaps the website using a HTML element that wasn't expected...

In [15]:
from transformers import pipeline, AutoTokenizer, AutoModel,LEDTokenizer, LEDForConditionalGeneration

trained_tokeniser = LEDTokenizer.from_pretrained("/kaggle/working/new_terms_model_4_epoch")
doc = dataset["test"]["document"][10]

    
tokenised_doc = trained_tokeniser(doc,truncation=True).input_ids # tokenise document
decoded_example = tokeniser.decode(tokenised_doc, skip_special_tokens=True) # convert tokenised document back to english


print(f'Example document length: {len(doc)}')
print(f'Tokenised document length {len(decoded_example)}')

Example document length: 33546
Tokenised document length 33545


Next, I want to look at the average number of text lost in the test set during the tokenisation process due to truncation.

In [16]:
trained_tokeniser = LEDTokenizer.from_pretrained("new_terms_model_4_epoch")

test_set = dataset["test"]["document"]

untokenised_lengths_arr = []
tokenised_lengths_arr = []

for doc in test_set:
    untokenised_length = len(doc)
    untokenised_lengths_arr.append(untokenised_length)
    
    tokenised_doc = trained_tokeniser(doc,truncation=True).input_ids
    decoded_example = tokeniser.decode(tokenised_doc, skip_special_tokens=True)
    tokenised_lengths_arr.append(len(decoded_example))
    

average_length_untokenised = sum(untokenised_lengths_arr) / len(untokenised_lengths_arr)
average_length_tokenised = sum(tokenised_lengths_arr) / len(tokenised_lengths_arr)

difference = abs(average_length_untokenised - average_length_tokenised)

print(f'The untokenised set had an average length of {round(average_length_untokenised)} characters ',
      f'compared to the average length of the tokenised documents of {round(average_length_tokenised)} characters ',
      f'for a difference of {round(difference)} characters between the two averages')

The untokenised set had an average length of 29213 characters  compared to the average length of the tokenised documents of 28561 characters  for a difference of 653 characters between the two averages


## Test Set examples

To evaluate this model, I will use the hugging Face pipeline to initialise the newly trained model.

Recall a test set was set aside and was not used in training or validation, this can be used to see how the model produces summaries on input data is has not yet seen.

To start, I want to see how the model summarises some of the examples in the test set, and manually observe the output.

In [17]:

# Initialise model from filesystem.
trained_model = LEDForConditionalGeneration.from_pretrained("/kaggle/working/new_terms_model_4_epoch")

# Use HuggingFace pipeline to pass in the model, tokeniser and tell the pipeline it's a summarisation task.
summarizer = pipeline("summarization", model=trained_model,tokenizer=trained_tokeniser)

# Pick out and tokenise an example from the test set
example = dataset["test"]["document"][0]
tokenised_example = trained_tokeniser(example,return_tensors="pt",truncation=True).input_ids

# See if any of the input text was truncated during tokenisation
print(f'length of example: {len(example)}')
tokenised_example_no_tensor = trained_tokeniser(example,truncation=True).input_ids
decoded_example = tokeniser.decode(tokenised_example_no_tensor, skip_special_tokens=True)
print(f'Length of tokenised example: {len(decoded_example)}')

# Generate a summary
outputs = trained_model.generate(tokenised_example)

# Observe output - skip special tokens such as <s> 
result = trained_tokeniser.decode(outputs[0], skip_special_tokens=True)

ground_truth_summary = dataset["test"]["summary"][0]

print(f'Ground truth summary: {ground_truth_summary}')
print(" ")
print(f'Produced model summary: {result}')

length of example: 18010
Length of tokenised example: 18000
Ground truth summary: You authorise the service to charge a credit card supplied on re-occurring basis. The service does not guarantee accuracy or reliability of the information provided. The service provider makes no warranty regarding uninterrupted, timely, secure or error-free service. The service is provided 'as is' and to be used at the users' sole risk. Invalidity of any portion of the Terms of Service does not entail invalidity of its remainder. The service can delete specific content without reason and may do it without prior notice. You agree to defend, indemnify, and hold the service harmless in case of a claim related to your use of the service. Failure to enforce any provision of the Terms of Service does not constitute a waiver of such provision
 
Produced model summary: The service is provided 'as is' and to be used at the users' sole risk. The service provider makes no warranty regarding uninterrupted, timely, s

In [18]:
# Pick out and tokenise an example from the test set
example = dataset["test"]["document"][10]
tokenised_example = trained_tokeniser(example,return_tensors="pt",truncation=True).input_ids

# See if any of the input text was truncated during tokenisation
print(f'length of example: {len(example)}')
tokenised_example_no_tensor = trained_tokeniser(example,truncation=True).input_ids
decoded_example = tokeniser.decode(tokenised_example_no_tensor, skip_special_tokens=True)
print(f'Length of tokenised example: {len(decoded_example)}')

# Generate a summary
outputs = trained_model.generate(tokenised_example)

# Observe output - skip special tokens such as <s> 
result = trained_tokeniser.decode(outputs[0], skip_special_tokens=True)

ground_truth_summary = dataset["test"]["summary"][10]

print(f'Ground truth summary: {ground_truth_summary}')
print(" ")
print(f'Produced model summary: {result}')

length of example: 33546
Length of tokenised example: 33545
Ground truth summary: This service may collect, use, and share location data. You agree not to submit libelous, harassing or threatening content. There is a date of the last update of the agreements. You are responsible for maintaining the security of your account and for the activities on your account. You authorise the service to charge a credit card supplied on re-occurring basis. You must report to the service any unauthorized use of your account or any breach of security. Copyright license limited for the purposes of that same service but transferable and sublicenseable. The service provider makes no warranty regarding uninterrupted, timely, secure or error-free service. If you offer suggestions to the service, they may use that without your approval or compensation, but they do not become the owner. You have a reduced time period to take legal action against the service. This service reserves the right to disclose your p

## Evaluating complete test set

Recall the test set put to one side earlier has not been seen at all by the model.

I will now show these to the model and calculate the ROUGE score on the produced summaries.

In [19]:
summaries = []
truths = []
count = 1

for item in dataset["test"]:
    print(f'Processing document {count} of {len(dataset["test"])}')
    document = item["document"]
    ground_truth_summary = item["summary"]
    
    tokenised_document = trained_tokeniser(document,return_tensors="pt",truncation=True).input_ids
    generated_summary = trained_model.generate(tokenised_document)
    decoded_summary = trained_tokeniser.decode(generated_summary[0], skip_special_tokens=True)
    
    summaries.append(decoded_summary)
    truths.append(ground_truth_summary)
    count += 1
    
rouge_score = rouge.compute(predictions=summaries,references=truths)
print(rouge_score)

Processing document 1 of 53
Processing document 2 of 53
Processing document 3 of 53
Processing document 4 of 53
Processing document 5 of 53
Processing document 6 of 53
Processing document 7 of 53
Processing document 8 of 53
Processing document 9 of 53
Processing document 10 of 53
Processing document 11 of 53
Processing document 12 of 53
Processing document 13 of 53
Processing document 14 of 53
Processing document 15 of 53
Processing document 16 of 53
Processing document 17 of 53
Processing document 18 of 53
Processing document 19 of 53
Processing document 20 of 53
Processing document 21 of 53
Processing document 22 of 53
Processing document 23 of 53
Processing document 24 of 53
Processing document 25 of 53
Processing document 26 of 53
Processing document 27 of 53
Processing document 28 of 53
Processing document 29 of 53
Processing document 30 of 53
Processing document 31 of 53
Processing document 32 of 53
Processing document 33 of 53
Processing document 34 of 53
Processing document 35 